**ALGORITHM 1: VL pretrained-M fine tuned with semi-disentangled outputs**

In [1]:
import requests
import os
import random
import json
import torch
import os
import kagglehub
import pickle
import math
import cv2

import numpy as np
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import torch.nn.functional as F

from PIL import Image
from collections import Counter
from sklearn.decomposition import PCA
from skimage.morphology import skeletonize
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import Dataset, DataLoader
from transformers import CLIPProcessor, CLIPModel, AutoModelForSeq2SeqLM, AutoTokenizer
from sklearn.model_selection import train_test_split


from torchvision import transforms as A

In [2]:
# path = kagglehub.dataset_download("albertozorzetto/carla-densely-annotated-driving-dataset")
# print("Path to dataset files:", path)

Dataset Creation (IMAGE form to EMBEDDING form)

In [2]:
# # Download the CARLA dataset using the Kaggle API
# dataset_name = "albertozorzetto/carla-densely-annotated-driving-dataset"
# destination_path = "/content/datasets"

# import os
# # Install Kaggle API
# !pip install kaggle --upgrade
# # Upload kaggle.json for authentication
# from google.colab import files
# uploaded = files.upload()
# # Move kaggle.json to the proper directory
# !mkdir -p ~/.kaggle
# !mv kaggle.json ~/.kaggle/
# !chmod 600 ~/.kaggle/kaggle.json
# # Verify Kaggle is set up correctly
# !kaggle datasets list
# !kaggle datasets download -d {dataset_name} -p {destination_path} --unzip
# print("Path to dataset files:", destination_path)

# import tarfile
# images_path = os.path.join(destination_path, "images")
# labels_path = os.path.join(destination_path, "labels")
# # Create directories for extracted files
# os.makedirs(images_path, exist_ok=True)
# os.makedirs(labels_path, exist_ok=True)
# # Extract and manage .tar files
# tar_files = [f for f in os.listdir(destination_path) if f.endswith('.tar')]
# for tar_file in tar_files:
#     tar_path = os.path.join(destination_path, tar_file)
#     print(f"Extracting {tar_path}...")
#     with tarfile.open(tar_path) as tar:
#         # Determine the folder to extract based on the tar file name
#         if "images" in tar_file.lower():
#             tar.extractall(path=images_path)
#         elif "labels" in tar_file.lower():
#             tar.extractall(path=labels_path)
#         else:
#             print(f"Skipping unknown .tar file: {tar_file}")
#     os.remove(tar_path)  # Optional: Remove the .tar file after extraction
# print("All .tar files have been extracted.")
# print(f"Images folder: {images_path}")
# print(f"Labels folder: {labels_path}")

In [2]:
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
semantic_label_classes = ["Traffic Sign", "Building", "Fence", "Other", "Pedestrian", "Pole", "Road Line", "Road", "Sidewalk", "Vegetation", "Car", "Wall", "Unlabeled"]

root = "datasets"
images_dir = f"{root}/images"
labels_dir = f"{root}/labels"
waypoint_file = 'dataset_waypoints_try.json'
batch_size = 2

skip = 100

*Saving the {way points} for passing to the LLM*

In [16]:
def generate_waypoints(segmentation_map, num_waypoints=10):
    road_mask = (segmentation_map == 90).astype(np.uint8)
    contours, _ = cv2.findContours(road_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not contours:
        return []
    largest_contour = max(contours, key=cv2.contourArea)
    binary_image = (road_mask > 0).astype(np.uint8)
    skeleton = skeletonize(binary_image).astype(np.uint8)
    y_coords, x_coords = np.where(skeleton > 0)
    path_points = np.column_stack((x_coords, y_coords))
    sampled_indices = np.linspace(0, len(path_points) - 1, num_waypoints, dtype=int)
    waypoints = path_points[sampled_indices]

    return waypoints.tolist()

In [18]:
def process_dataset(images_dir, labels_dir, output_dir, num_waypoints=10):
    os.makedirs(output_dir, exist_ok=True)
    dataset_waypoints = []

    for i in range(0, 28):
        k = "0" if i <= 9 else ""
        curr_folder_labels = f"{labels_dir}/Video_0{k}{i}"
        labels_path = [file for file in os.listdir(curr_folder_labels) if file.endswith(".png")]
        labels_path.sort()
        labels_path = labels_path[::skip]

        for label_file in labels_path:
            label_path = os.path.join(curr_folder_labels, label_file)
            label = Image.open(label_path)
            label = label.convert("L")
            label_np = np.array(label)
            waypoints = generate_waypoints(label_np, num_waypoints=num_waypoints)
            output_label_path = os.path.join(output_dir, label_file)
            dataset_waypoints.append({
                "image": output_label_path,
                "waypoints": waypoints
            })
            # break

        with open(os.path.join(output_dir, waypoint_file), 'a') as wf:
            json.dump(dataset_waypoints, wf)
            wf.write("\n")
            # break

        if(i<27):
            print(f"\rProgress: {((i + 1) / 0.27):.2f} %", end = "")
        # break

    print(f"\nDone")

In [ ]:
process_dataset(images_dir, labels_dir, root, num_waypoints=10)

*Fetching Data from images folder and dataset_waypoints.json*

In [51]:
class SegmentationDataset(Dataset):
    def __init__(self, image_dir, labels_dir):
        self.data = []
        ei = []
        cw = []

        for i in range(0, 28):
            k = "0" if i <= 9 else ""
            curr_folder_images = f"{images_dir}/Video_0{k}{i}"
            images_path = [file for file in os.listdir(curr_folder_images) if file.endswith(".png")]
            images_path.sort()
            images_path = images_path[::skip]

            for image_file in images_path:
                image_path = os.path.join(curr_folder_images, image_file)
                image = Image.open(image_path)
                image_embeddings = processor(text=semantic_label_classes, images=image, return_tensors="pt", padding=True)
                ei.append(image_embeddings)

            with open(os.path.join(root, waypoint_file), "r") as wf:
                content = wf.read()
                data_list = content.splitlines()
                for data in data_list:
                    data = json.loads(data)
                    for i in data:
                        corresponding_waypoints = i['waypoints']
                        cw.append(corresponding_waypoints)
                        # print(corresponding_waypoints)
                        # break
                    # break
            for e,c in zip(ei, cw):
                self.data.append((e,c))
                # break
            # break
        print(self.data[0][1])
    
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        embedding_inputs, embedding_labels = self.data[idx]
        return embedding_inputs, embedding_labels

In [ ]:
# Split indices for train and validation
dataset = SegmentationDataset(images_dir, labels_dir)
dataloader = DataLoader(dataset, batch_size = batch_size, shuffle = True, pin_memory=True)

train_indices, val_indices = train_test_split(range(len(dataset)), test_size=0.2, random_state=42)
train_dataset = torch.utils.data.Subset(dataset, train_indices)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True)

val_dataset = torch.utils.data.Subset(dataset, val_indices)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, pin_memory=True)

In [ ]:
from itertools import chain

for i, j in dataset:
    flattened_j = list(chain.from_iterable(j))
    avg = sum(flattened_j) / len(flattened_j)
    print(avg)


In [50]:
def emb_to_GPU(emb):
  emb = {key: value.to(device) for key, value in emb.items()}
  return emb

In [6]:
def un_list(dic):
    for key in dic.keys():
        dic[key] = dic[key][0]
    return dic

In [10]:
def normalize(pixel_values):
    mini = pixel_values.min()
    maxi = pixel_values.max()
    pixel_values = (pixel_values - mini) / (maxi - mini)
    return pixel_values

In [2]:
def to_float(val):
    return val.float()

OUR MODEL

In [3]:
class WaypointModel(nn.Module):
    def __init__(self):
        super(WaypointModel, self).__init__()
        self.embedding_fc = nn.Linear(13 * 4, 256)
        self.attention_fc = nn.Linear(13 * 4, 256)
        self.pixel_fc = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        self.fc1 = nn.Linear(256 + 256 + 64 * 56 * 56, 512)
        self.fc2 = nn.Linear(512, 20)

    def forward(self, input_ids, attention_mask, pixel_values):
        
        batch_size = input_ids.size(0)

        assert input_ids.size(1) == 13 and input_ids.size(2) == 4, "Expected input_ids shape to be (batch_size, 13, 4)"
        embedding_out = self.embedding_fc(input_ids.view(batch_size, -1))
        assert attention_mask.size(1) == 13 and attention_mask.size(2) == 4, "Expected attention_mask shape to be (batch_size, 13, 4)"
        attention_out = self.attention_fc(attention_mask.view(batch_size, -1))
        pixel_values = pixel_values.squeeze(1)
        
        pixel_out = self.pixel_fc(pixel_values)
        pixel_out = pixel_out.view(batch_size, -1)
        
        combined = torch.cat((embedding_out, attention_out, pixel_out), dim=1)
        x = torch.relu(self.fc1(combined))
        output = self.fc2(x)

        return output.view(-1, 10, 2)

*--training--*

In [112]:
# from google.colab import drive
# drive.mount('/content/drive')

In [114]:
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# print(f"Using device: {device}")
# model = model.to(device)

In [158]:
model = WaypointModel()
criterion = nn.MSELoss()

In [71]:
optimizer = optim.Adam(model.parameters(), lr=1e-2)

In [56]:
def compute_accuracy(outputs, targets):
  threshold = targets.mean() * 0.01
  # print("outputs:", outputs)
  # print("targets:", targets)
  MSE = torch.sqrt(torch.sum((outputs - targets) ** 2, dim=-1))
  accurate_predictions = (MSE <= threshold).float()
  accuracy = accurate_predictions.mean().item()
  # print("MSE:", MSE)
  # print("acc preds:", accurate_predictions)
  # print("acc:", accuracy)
  return accuracy

In [4]:
def model_output_generation(emb, model):
    input_ids = to_float(emb['input_ids'])
    attention_mask = to_float(emb['attention_mask'])
    pixel_values = emb['pixel_values'].squeeze(1)

    if input_ids.size(1) != 13 or input_ids.size(2) != 4:
        input_ids = input_ids.view(-1, 13, 4)
    if attention_mask.size(1) != 13 or attention_mask.size(2) != 4:
        attention_mask = attention_mask.view(-1, 13, 4)

    outputs = model(input_ids, attention_mask, pixel_values)
    return outputs

In [54]:
def wayp_to_targets(way_p):
    return to_float(torch.stack([torch.stack(pair, dim = 0) for pair in way_p]).permute(2,0,1))

In [133]:
def training(model, dataloader, optimizer, criterion):
    model.train()
    running_loss = 0.0
    running_accuracy = 0.0

    for emb, way_p in dataloader:        
        # print(way_p)
        target = wayp_to_targets(way_p)
        optimizer.zero_grad()

        outputs = model_output_generation(emb, model)
        loss = criterion(outputs, target)
        running_loss += loss.item()

        accuracy = compute_accuracy(outputs, target)
        running_accuracy += accuracy
        
        loss.backward()
        optimizer.step()

    return running_loss / len(dataloader), running_accuracy / len(dataloader)

In [167]:
def validation(model, dataloader, optimizer, criterion):
    model.eval()
    running_loss = 0.0
    running_accuracy = 0.0

    with torch.no_grad():
        for i, (emb, way_p) in enumerate(dataloader):
            target = wayp_to_targets(way_p)

            outputs = model_output_generation(emb, model)
            loss = criterion(outputs, target)
            running_loss += loss.item()

            accuracy = compute_accuracy(outputs, target)
            running_accuracy += accuracy

            print(f"\r{outputs.mean()}       ", end = "")
            if(outputs.mean() < 500 and outputs.mean() > 300):
                running_loss = 350 * len(dataloader)
            break

    return running_loss / len(dataloader), running_accuracy / len(dataloader)

In [168]:
model = WaypointModel()
criterion = nn.MSELoss()
num_epochs = 50
lr = 1e-1

In [ ]:
for i in range(0,4):
  lr = lr / 10
  optimizer = optim.Adam(model.parameters(), lr=lr)
  prev_train_loss = float('inf')
  prev_val_loss = float('inf')
  # patience = 5
  best_model = None
  print(f"Enhancing Learning rate at: {lr}")

  for epoch in range(num_epochs):
    best_model = model
    # if(patience == 0):
    #   break
    
    train_loss, train_accuracy = training(model, train_loader, optimizer, criterion)
    val_loss, val_accuracy = validation(model, val_loader, optimizer, criterion)

    if(val_loss < 400):
      break
    
    if(train_loss > (prev_train_loss * 0.95) or (val_loss > (prev_val_loss * 0.95))):
      model = best_model
      # patience -= 1
    else:
      prev_train_loss = train_loss
      prev_val_loss = val_loss
      best_model = model
      # patience = 5
      # torch.save(model.state_dict(), '/content/drive/MyDrive/my_model.pth')
    
    # print(f"\rEpoch {epoch + 1}/{num_epochs}, train-Loss: {train_loss:.1f}, train-Accuracy: {train_accuracy:.1f}, val-Loss: {val_loss:.1f}, val-Accuracy: {val_accuracy:.1f}", end = "")#, patience: {patience}", end = "")
  print("\n")
  break

In [170]:
driver_model = model

In [199]:
torch.save(best_model.state_dict(), 'best_model.pth')

**CarLA simulation**

In [5]:
import carla
import random
import time
import kagglehub
import os
import torch
import requests

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from transformers import SegformerForSemanticSegmentation, SegformerConfig
from transformers import AutoImageProcessor, SegformerForSemanticSegmentation
from PIL import Image
from transformers import CLIPProcessor, CLIPModel

In [6]:
driver_model = WaypointModel()
driver_model.load_state_dict(torch.load('best_model.pth'))

C:\Users\devas\AppData\Local\Temp\ipykernel_5964\357336458.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  driver_model.load_state_dict(torch.load('best_model.pth'))


<All keys matched successfully>

In [7]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
semantic_label_classes = ["Traffic Sign", "Building", "Fence", "Other", "Pedestrian", "Pole", "Road Line", "Road", "Sidewalk", "Vegetation", "Car", "Wall", "Unlabeled"]

In [8]:
client = carla.Client('localhost', 2000)
world = client.get_world()
blueprint_library = world.get_blueprint_library()

In [9]:
def remove_all_actors():
    actors = world.get_actors()
    for actor in actors:
        if 'vehicle' in actor.type_id:
            actor.destroy()
    world.tick()
    print("previous ones destroyed")

In [73]:
remove_all_actors()

previous ones destroyed


In [74]:
ego_blueprint = blueprint_library.filter('*model3')[0]
spawn_points = world.get_map().get_spawn_points()[0]
ego = world.spawn_actor(ego_blueprint, spawn_points)

camera_bp = blueprint_library.find('sensor.camera.rgb')
camera_bp.set_attribute('image_size_x', '1920')
camera_bp.set_attribute('image_size_y', '1080')
camera_bp.set_attribute('fov', '110')
camera_transform = carla.Transform(carla.Location(x = 1.5, z = 2.4))
camera = world.spawn_actor(camera_bp, camera_transform, attach_to=ego)

In [57]:
def EGO_relative_view(EGO):
    view = EGO.get_transform().location + carla.Location(x=0, y=0, z=10)
    return view

In [58]:
def set_cam_to_EGO(EGO):
    spectator = world.get_spectator()
    vehicle_transform = EGO.get_transform()
    spectator.set_transform(
        carla.Transform(
            EGO_relative_view(EGO),
            carla.Rotation(pitch = -70, yaw = vehicle_transform.rotation.yaw, roll = 0)
        )
    )

*controlling based on model*

In [59]:
def apply_action(vehicle, action):
    vehicle.apply_control(carla.VehicleControl(
        throttle=action['throttle'],
        steer=action['steer'],
        brake=action['brake']
    ))

In [75]:
def steering(x, intended_x):
    towards = x - intended_x
    steer = towards * 0.0002
    steer = max(-1, min(steer, 1))
    return steer

In [76]:
def TandB(y_mid, intended_y_mid, curr_y = 25):
    if(intended_y_mid <= curr_y):
        return 0, 1
    intended_y_mid = intended_y_mid if intended_y_mid < y_mid else y_mid
    return intended_y_mid / (4 * y_mid), 0

In [17]:
def process_image(image):
    array = np.frombuffer(image.raw_data, dtype=np.uint8)
    array = array.reshape((image.height, image.width, 4))
    return Image.fromarray(array[:, :, :3])

In [66]:
def image_callback(data):
    sensor_data["image"] = process_image(data)
sensor_data = {"image": None}

In [77]:
set_cam_to_EGO(ego)

In [23]:
def normalize(way_p, top_w,top_h):
    mini_f = float('inf')
    maxi_f = -float('inf')
    for i in way_p:
        if(maxi_f < i[0]):
            maxi_f = i[0]
        if(mini_f > i[0]):
            mini_f = i[0]
        
    mini_s = float('inf')
    maxi_s = -float('inf')
    for i in way_p:
        if(maxi_s < i[1]):
            maxi_s = i[1]
        if(mini_s > i[1]):
            mini_s = i[1]
        
    for i in way_p:
        i[0] = (i[0] - mini_f) / (maxi_f - mini_f)
        i[1] = (i[1] - mini_s) / (maxi_s - mini_s)
        i[0] *= top_h
        i[1] *= top_w

    return way_p

In [78]:
try:
    camera.listen(image_callback)
    while True:
        world.wait_for_tick()
        image = sensor_data['image']
        if image is not None:
            emb = processor(text=semantic_label_classes, images=image, return_tensors="pt", padding=True)
            outputs = model_output_generation(emb, driver_model)

            h,w = image.size
            x_mid = h/2
            y_mid = w/2
            control = carla.VehicleControl()
            waypoints = outputs.detach().numpy()

            ego_x = x_mid
            ego_y = 25

            new_wp = normalize(waypoints[0], h,w)
            intended_x_mid = np.mean([point[0] for point in new_wp])
            intended_y_mid = np.mean([point[1] for point in new_wp])

            steer = steering(ego_x, intended_x_mid)
            throttle, brake = TandB(y_mid, intended_y_mid, ego_y)
            
            control.steer = steer
            control.throttle = throttle
            control.brake = brake
            ego.apply_control(control)

            # print(intended_x_mid, x_mid)
            
            # plt.figure(figsize = (h/100, w/100))
            # plt.scatter(x_coords, y_coords, color='r', label='Waypoints')
            # x_coords = [point[0] for point in new_wp]
            # mean_x = np.mean(x_coords)
            # y_coords = [point[1] for point in new_wp]
            # mean_y = np.mean(y_coords)
            # plt.scatter(intended_x_mid, mean_y, color='b')
            # plt.scatter(mean_x, 0, color='g')
            # plt.axvline(x=mean_x, color='g', linestyle='--', label='X-axis Center')
            # plt.grid(True)
            # plt.show()
            
            # action = get_action(image)
            # apply_action(ego, action)
            
            break
finally:
    camera.stop()

In [ ]:
error here :)

*-- documentation part --*

In [ ]:
spectator = world.get_spectator()
transform = spectator.get_transform()
location = transform.location
rotation = transform.rotation
spectator.set_transform(carla.Transform())

In [ ]:
traffic_lights = world.get_actors().filter('traffic.traffic_light')
for traffic_light in traffic_lights:
    traffic_light.set_state(carla.TrafficLightState.Green)

In [ ]:
set_cam_to_EGO()
on_collision(EGO)

In [ ]:
vehicles = [actor for actor in world.get_actors() if 'vehicle' in actor.type_id]

for vehicle in vehicles:
    collision_sensor = world.spawn_actor(
        blueprint_library.find('sensor.other.collision'),
        carla.Transform(),
        attach_to=vehicle
    )
    collision_sensor.listen(lambda event, vehicle=vehicle: on_collision(vehicle, event))
    # collision_sensor.listen(lambda event: on_collision(event.actor))

In [ ]:
# EGO.set_autopilot(True)

# while True:
#     set_cam_to_EGO()
#     time.sleep(1/240)

In [ ]:
vehicle_transform = EGO.get_transform()

spectator.set_transform(
    carla.Transform(
        EGO_relative_view(),
        carla.Rotation(pitch = -30, yaw = vehicle_transform.rotation.yaw, roll = 0)
    )
)

time.sleep(0.05)

In [ ]:
# the recording function
def save_details(image):
    image.save_to_disk('dataset/out/%06d.png' % image.frame)

    metadata = {
        'frame': image.frame,
        'timestamp': image.timestamp,
        'width': image.width,
        'height': image.height,
        'camera_transform': camera.get_transform(),
        'camera_intrinsics': camera_bp.get_attribute('fov').as_dict()
    }

    with open(f'dataset/out/{image.frame:06d}_metadata.json', 'w') as f:
        json.dump(metadata, f)
        
    depth_camera_bp = world.get_blueprint_library().find('sensor.camera.depth')
    depth_camera = world.spawn_actor(depth_camera_bp, camera_init_trans, attach_to=EGO)

    def save_depth_image(depth_image):
        depth_image.save_to_disk(f'dataset/out/{depth_image.frame:06d}_depth.png')

    depth_camera.listen(save_depth_image)

camera_init_trans = carla.Transform(carla.Location(z=1.5))
camera_bp = world.get_blueprint_library().find('sensor.camera.rgb')
camera = world.spawn_actor(camera_bp, camera_init_trans, attach_to=EGO)

camera.listen(save_details)

In [ ]:
# blueprint = properties for some configuration
collision_sensor_bp = blueprint_library.find('sensor.other.collision')
vehicle_bp = random.choice(blueprint_library.filter('vehicle.*.*'))

In [ ]:
# Find the blueprint of the sensor.
blueprint = world.get_blueprint_library().find('sensor.camera.rgb')
# Modify the attributes of the blueprint to set image resolution and field of view.
blueprint.set_attribute('image_size_x', '1920')
blueprint.set_attribute('image_size_y', '1080')
blueprint.set_attribute('fov', '110')
# Set the time in seconds between sensor captures
blueprint.set_attribute('sensor_tick', '1.0')

In [ ]:
transform = carla.Transform(relative_view)
sensor = world.spawn_actor(blueprint, transform, attach_to=EGO)

In [ ]:
# sensor.listen(lambda data: do_something(data))

# This collision sensor would print everytime a collision is detected. 
def callback(event):
    for actor_id in event:
        vehicle = world_ref().get_actor(actor_id)
        print('Vehicle too close: %s' % vehicle.type_id)

sensor.listen(callback)